In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import pickle
import gzip
import time
from warnings import filterwarnings
filterwarnings('ignore')
from _utils import utils, transforming, filling

data = ['../BigContest_data/' + file for file in os.listdir('../BigContest_data')]

In [4]:
# 기본 제공 데이터
user_spec = pd.read_csv(data[0])
log_data = pd.read_csv(data[1])
loan_result = pd.read_csv(data[2])

In [ ]:
print('user_spec')
print(tuple(map(utils.split_comma,user_spec.shape)))
display(user_spec.head())
print('log_data')
print(tuple(map(utils.split_comma,log_data.shape)))
display(log_data.head())
print('loan_result')
print(tuple(map(utils.split_comma,loan_result.shape)))
display(loan_result.head())

# 누락 데이터 제거
모든 결측치를 제거하는 것이 아니라 주최측 QnA에 따라 의미 없는 정보를 제거
- loan_result에는 있는데 user_spec에는 없는 application_id는 제외
- loan_result의 loan_rate와 loan_limit이 결측치인 경우 제외

--> 다음의 누락데이터 제거를 사용하려했으나 2번 Feature Extraction에서 더 많은 변수를 추출하기 위해 원본 loan_result를 사용하기로 한다.(다음의 일부 제거된 데이터를 저장하지 않는다.)

In [6]:
# loan_result에는 있는데 user_spec에는 없는 application_id는 제외
drop_application = np.setdiff1d(loan_result['application_id'].unique(),user_spec['application_id'].unique())
drop_cond = loan_result['application_id'].isin(drop_application)
loan_result = loan_result[drop_cond==False]

# loan_result의 loan_rate와 loan_limit이 결측치인 경우 제외
loan_result = loan_result.dropna(subset=['loan_rate','loan_limit'], how='any')

In [ ]:
# 로그데이터는 유실 가능, loan_result와 user_spec 위주로 살피면 됨
# 다음은 그냥 참고용
print(f"""log_data에는 있는데 user_spec 데이터에는 없는 user : {len(np.setdiff1d(log_data['user_id'],user_spec['user_id']))}명,
user_spec에는 있는데 log_data 데이터에는 없는 user : {len(np.setdiff1d(user_spec['user_id'],log_data['user_id']))}명""")

In [8]:
# user_id와 application_id는 매칭(dictionary)
temp = user_spec.groupby('user_id').agg({'application_id':list})
user_application_dict = dict(zip(temp.index, temp['application_id']))
application_user_dict = dict(zip(user_spec['application_id'], user_spec['user_id']))

# 결측치 보간
- 같은 user_id인 application_id에 대해 어떤 application_id는 값이 비워져있는데 어떤 application_id는 값이 채워져있다면 이를 토대로 결측값을 채울 수 있다.<br>
    ex) user_id=1, application_id=1, gender=1 인 행이 있고 user_id=1, application_id=2, gender=NaN 인 행이 있다면 이 결측치는 1로 채울 수 있다.
    
## birth_year, gender
- 최빈값을 이용하기로함

In [ ]:
user_spec[['gender','birth_year']].isnull().sum()

In [ ]:
is_gender_col_complete = utils.is_unique(user_spec, 'gender')
# nan으로만 구성된 user_id는 포함되지 않는다.
is_gender_col_complete[is_gender_col_complete > 1].head()

In [ ]:
# gender=1라는 정보가 실려있으므로 gender의 결측치 채워넣기 가능
display(user_spec.query('user_id==52').iloc[:,:5])
# gender=2라는 정보가 실려있으므로 gender의 결측치 채워넣기 가능
display(user_spec.query('user_id==1318').iloc[:,:5])
# 이런것은 채우지 못함
display(user_spec.query('user_id==877748').iloc[:,:5])

In [16]:
user_spec_filled = filling.fillna(user_spec, 'gender', 'mode')
user_spec_filled = filling.fillna(user_spec_filled, 'birth_year', 'mode')

In [ ]:
# 위에서 확인한 52번과 1318번,877748번 고객 정보를 통해 잘 채워진것을 확인할 수 있음
display(pd.concat([user_spec_filled.query('user_id==52').iloc[:,:5],
                   user_spec_filled.query('user_id==1318').iloc[:,:5],
                   user_spec_filled.query('user_id==877748').iloc[:,:5]],axis=0))

In [ ]:
user_spec_filled[['gender','birth_year']].isnull().sum()

## credit_score, yearly_income
- 시간에 따른 보간을 이용하기로 한다. 단 신용점수의 보간값이 일의 자리가 생기는 것을 막기 위해 반올림한다.

In [ ]:
user_spec[['credit_score','yearly_income']].isnull().sum()

In [ ]:
# 예시
display(user_spec.loc[user_spec['user_id']==387038,['user_id','gender','insert_time','credit_score','yearly_income']])

In [21]:
start = time.time()
user_spec_filled = filling.fillna(user_spec_filled, 'credit_score', 'interpolate')
user_spec_filled = filling.fillna(user_spec_filled, 'yearly_income', 'interpolate')
print((time.time() - start)/60,'분')

4.202482493718465 분


In [ ]:
# 예시 데이터 fill
display(user_spec_filled.loc[user_spec_filled['user_id']==387038,['user_id','gender','insert_time','credit_score','yearly_income']])

In [ ]:
user_spec_filled[['credit_score','yearly_income']].isnull().sum()

## income_type, employment_type, houseown_type
- 명목형 변수는 이전값(주위값)으로 채우기로 한다.

In [ ]:
user_spec[['income_type','employment_type','houseown_type']].isnull().sum()

In [ ]:
# 예시
display(user_spec.query('user_id==88930')[['application_id','user_id','insert_time','income_type','employment_type','houseown_type']])

In [39]:
# purpose는 실제로 그때그때 다를 수 있으므로 채워넣지 않는다.
# 'income_type','employment_type','houseown_type'는 가까운 시간 사이에는 변하지 않을 가능성이 높기 때문에 채워넣는다.
start = time.time()
user_spec_filled = filling.fillna(user_spec_filled,'income_type','order' )
user_spec_filled = filling.fillna(user_spec_filled,'employment_type','order' )
user_spec_filled = filling.fillna(user_spec_filled,'houseown_type','order' )
print((time.time() - start)/60,'분')

0.02245872418085734 분


In [ ]:
# 예시 데이터 fill
display(user_spec_filled.query('user_id==88930')[['application_id','user_id','insert_time','income_type','employment_type','houseown_type']])

In [ ]:
user_spec_filled[['income_type','employment_type','houseown_type']].isnull().sum()

## company_enter_month
- mode를 취하되 입사년도 자체보다는 입사 후 지난 기간을 고려하기 위해 추후 더 처리를 해주도록 한다.

In [42]:
start = time.time()
user_spec_filled = filling.fillna(user_spec_filled,'company_enter_month','mode' )
print((time.time() - start)/60,'분')

0.5693900307019552 분


## personal_rehabilitation_yn, personal_rehabilitation_complete_yn
- rehabilitation==nan
    - 다른 application이 있다 → rehabilitation과 complete 모두 이전값으로 채우기
    - 다른 application이 없다 → 0
- rehabilitation==0 → complete 2(해당없음)
- rehabilitation==1 → complete 결측치 x (결측치 처리 필요 없음)

In [54]:
df_consider = user_spec[['user_id','insert_time','personal_rehabilitation_yn', 'personal_rehabilitation_complete_yn']]

In [ ]:
(df_consider['personal_rehabilitation_yn'].astype('str') + '_' + df_consider['personal_rehabilitation_complete_yn'].astype('str')).value_counts(dropna=False)

In [57]:
def fill_reha_compl_frst(df):
    if (df['personal_rehabilitation_yn']==0) & (pd.isna(df['personal_rehabilitation_complete_yn'])):
        complete = 2
    else:
        complete = df['personal_rehabilitation_complete_yn']
    return complete
df_consider['personal_rehabilitation_complete_yn'] = df_consider.apply(fill_reha_compl_frst,axis=1)

In [ ]:
(df_consider['personal_rehabilitation_yn'].astype('str') + '_' + df_consider['personal_rehabilitation_complete_yn'].astype('str')).value_counts(dropna=False)

In [64]:
# 주의 : 결측치가 많아 시간이 오래걸림(30분 가량)
df_consider_filled = filling.fillna(df_consider, 'personal_rehabilitation_yn', 'order')
df_consider_filled = filling.fillna(df_consider_filled, 'personal_rehabilitation_complete_yn', 'order')

In [ ]:
(df_consider_filled['personal_rehabilitation_yn'].astype('str') + '_' + df_consider_filled['personal_rehabilitation_complete_yn'].astype('str')).value_counts(dropna=False)

In [68]:
null_user_id = df_consider_filled.loc[df_consider_filled['personal_rehabilitation_yn'].isnull(),'user_id']

In [ ]:
# 같은 user_id의 다른 application_id에서 참조할 값이 없는 고객들이니 0으로 채운다.
df_consider_filled.loc[df_consider_filled['user_id'].isin(null_user_id), 'personal_rehabilitation_yn'].sum()

In [71]:
df_consider_filled['personal_rehabilitation_yn'] = df_consider_filled['personal_rehabilitation_yn'].fillna(0)
df_consider_filled['personal_rehabilitation_complete_yn'] = df_consider_filled['personal_rehabilitation_complete_yn'].fillna(2)


In [ ]:
df_consider_filled.isnull().sum()

In [75]:
user_spec_filled.loc[:,['user_id','insert_time','personal_rehabilitation_yn', 'personal_rehabilitation_complete_yn']] = df_consider_filled

## desired amount
loan_result에 is_applied = 1이 있으면, 그 상품 금액만 모아서  desired_amount 넣으려는 전략을 취하려했으나 desired_amount가 null인 application_id가 loan_result에는 없음

In [84]:
desired_amount_null_app_id = user_spec_filled.loc[user_spec_filled['desired_amount'].isnull(),'application_id'].values
loan_result[loan_result['application_id'].isin(desired_amount_null_app_id)]

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied


## exisiting_loan_cnt, existing_loan_amt
- 이전값으로 

In [89]:
df_consider = user_spec[['application_id','user_id','insert_time','existing_loan_cnt', 'existing_loan_amt']]

In [ ]:
null_cond1 = df_consider['existing_loan_cnt'].isnull()
null_cond2 = df_consider['existing_loan_amt'].isnull()

df_consider_null = df_consider.loc[null_cond1 | null_cond2,:]
display(df_consider_null.head())
print(df_consider_null.shape)

In [114]:
df_consider_null = df_consider[df_consider['user_id'].isin(df_consider_null['user_id'].values)]

In [119]:
# 아마 같은 user_id에서 참조할게 없어보임 -> 보간 함수 적용해도 소용 없을듯

df_consider_null.shape

(313774, 5)

In [123]:
start = time.time()
user_spec_filled = filling.fillna(user_spec_filled,'existing_loan_cnt','order')
print((time.time()-start)/60,'분')

6.705570793151855 분


In [124]:
# 실제로 채워지지 않음, existing_loan_amt 역시 마찬가지
user_spec_filled['existing_loan_cnt'].isnull().sum()

198556

In [127]:
display(pd.concat([user_spec.isnull().sum(), user_spec_filled.isnull().sum()],axis=1).rename(columns={0:'origin',1:'after fill'}))

,origin,after fill
application_id,0,0
user_id,0,0
birth_year,12961,9724
gender,12961,9724
insert_time,0,0
credit_score,105115,87524
yearly_income,90,19
income_type,85,18
company_enter_month,171760,114112
employment_type,18,18


# user_spec의 purpose가 한글, 영어 섞여있으니 통일

In [ ]:
user_spec_filled['purpose'].unique()

In [ ]:
mapping = {'생활비':'LIVING','대환대출':'SWITCHLOAN','기타':'ETC','투자':'INVEST',
          '사업자금':'BUSINESS','자동차구입':'BUYCAR','전월세보증금':'HOUSEDEPOSIT','주택구입':'BUYHOUSE'}
user_spec_filled['purpose'].replace(mapping).unique()

In [178]:
user_spec_filled['purpose'] = user_spec_filled['purpose'].replace(mapping)

# user_spec의 employment_type, houseown_type

In [ ]:
# 다음 값은 한글이므로 encoding 문제를 대비해 영어로 바꿔놓기로 한다.
user_spec_filled['employment_type'].unique()

In [ ]:
user_spec_filled['houseown_type'].unique()

In [201]:
employment_type_dict = {'기타':'etc','정규직':'regular_worker','계약직':'contract_worker','일용직':'daily_worker'}
user_spec_filled['employment_type'] = user_spec_filled['employment_type'].replace(employment_type_dict)

In [206]:
houseown_type_dict = {'자가':'own','기타가족소유':'family','전월세':'rent','배우자':'spouse'}
user_spec_filled['houseown_type'] = user_spec_filled['houseown_type'].replace(houseown_type_dict)

# company_enter_month 형식 통일

In [ ]:
# 다음과 같이 맞지 않음
user_spec_filled.loc[[0,1394211],'company_enter_month']

In [194]:
user_spec_filled['company_enter_month'] = user_spec_filled['company_enter_month'].map(utils.make_date_format)

In [207]:
user_spec_filled.to_csv('../preprocessed/user_spec_filled.csv',index=False)

# train, test set 나누기

In [129]:
# 3,4,5월 / 6월 기준으로 분할
# 분석 전 과정은 train set을 이용하되, 같은 유저의 정보를 이용해 채워넣을 수 있는 값을 채워넣는 결측치 보간은 전 user_spec을 이용해 진행하였다.
user_spec_train, user_spec_test = utils.split_train_test(user_spec_filled, 'insert_time')
loan_result_train, loan_result_test = utils.split_train_test(loan_result, 'loanapply_insert_time')
log_data_train, log_data_test = utils.split_train_test(log_data, 'timestamp')

# log data에 isApplied 끼워넣기

In [131]:
# log_data는 user_id, loan_result는 user_id기준으로 되어있으니 mapping하는 사전을 정의
temp = user_spec.groupby('user_id').agg({'application_id':list})
user_application_dict = dict(zip(temp.index, temp['application_id']))
application_user_dict = dict(zip(user_spec['application_id'], user_spec['user_id']))

In [ ]:
# 분석은 log_data_train으로 한다.
# 어파치 6월 데이터인 loan_result_test에는 is_applied가 모두 null이기 때문에 관계 파악 불가
loan_result_test['is_applied'].unique()

In [132]:
applied_Y = loan_result_train.query('is_applied==1')
applied_Y['user_id'] = applied_Y['application_id'].map(application_user_dict)
applied_Y = applied_Y[['user_id','loanapply_insert_time','is_applied']]

In [135]:
# mp_os, mp_app_version, date_cd는 중요 정보 같지도 않고 같이 끼워넣을 방법이 없어서 버림
applied_Y['is_applied']='is_applied_Y'
applied_Y = applied_Y.rename(columns={'loanapply_insert_time':'timestamp','is_applied':'event'})
concat_log = pd.concat([log_data_train[['user_id','event','timestamp']],applied_Y],axis=0)
concat_log = concat_log.sort_values(by=['user_id','timestamp'])

In [ ]:
# 예시 
display(log_data_train.query('user_id==65'))
display(applied_Y.query('user_id==65'))
display(concat_log.query('user_id==65'))

In [175]:
# 중간 결과 저장
concat_log.to_csv('../preprocessed/concat_log.csv',index=False)

## 해당 데이터들은 따로 저장하지 않고 필요하면 정의된 함수로 그때그때 만들어 사용한다.
# user_spec_train.to_csv('../preprocessed/user_spec_train.csv',index=False)
# user_spec_test.to_csv('../preprocessed/user_spec_test.csv',index=False)
# loan_result_train.to_csv('../preprocessed/loan_result_train.csv',index=False)
# loan_result_test.to_csv('../preprocessed/loan_result_test.csv',index=False)
# log_data_train.to_csv('../preprocessed/log_data_train.csv',index=False)
# log_data_test.to_csv('../preprocessed/log_data_test.csv',index=False)
